In [1]:
import opik

opik.configure(use_local=True, url='http://trainingvm:5173')

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Create dataset
import opik
import pandas as pd

client = opik.Opik()

# Create dataset
dataset = client.get_or_create_dataset(name="HaluEval", description="HaluEval dataset")

# Insert items into dataset
df = pd.read_parquet(
    "hf://datasets/pminervini/HaluEval/general/data-00000-of-00001.parquet"
)

df = df.sample(n=50, random_state=42)

dataset_records = [
    {
        "input": x["user_query"],
        "llm_output": x["chatgpt_response"],
        "expected_hallucination_label": x["hallucination"],
    }
    for x in df.to_dict(orient="records")
]

dataset.insert(dataset_records)

OPIK: Created a "HaluEval" dataset at http://trainingvm:5173/api/v1/session/redirect/datasets/?dataset_id=0195c7f9-76bd-7048-865e-2d8e3a7ef6c8&path=aHR0cDovL3RyYWluaW5ndm06NTE3My9hcGkv.
c:\demonstrations\mlops\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from opik.evaluation.metrics import Hallucination, Equals
from opik.evaluation import evaluate
from opik import Opik
from opik.evaluation.metrics.llm_judges.hallucination.template import generate_query
from typing import Dict


# Define the evaluation task
def evaluation_task(x: Dict):
    metric = Hallucination()
    try:
        metric_score = metric.score(input=x["input"], output=x["llm_output"])
        hallucination_score = metric_score.value
        hallucination_reason = metric_score.reason
    except Exception as e:
        print(e)
        hallucination_score = None
        hallucination_reason = str(e)

    return {
        "hallucination_score": "yes" if hallucination_score == 1 else "no",
        "hallucination_reason": hallucination_reason,
    }


# Get the dataset
client = Opik()
dataset = client.get_dataset(name="HaluEval")

# Define the scoring metric
check_hallucinated_metric = Equals(name="Correct hallucination score")

# Add the prompt template as an experiment configuration
experiment_config = {
    "prompt_template": generate_query(
        input="{input}", context="{context}", output="{output}", few_shot_examples=[]
    )
}

res = evaluate(
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[check_hallucinated_metric],
    experiment_config=experiment_config,
    scoring_key_mapping={
        "reference": "expected_hallucination_label",
        "output": "hallucination_score",
    },
)

Evaluation: 100%|██████████| 50/50 [00:18<00:00,  2.71it/s]


╭─ HaluEval (50 samples) ───────────────────╮
│                                           │
│ Total time:        00:00:18               │
│ Number of samples: 50                     │
│                                           │
│ Correct hallucination score: 0.7800 (avg) │
│                                           │
╰───────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=263986;http://trainingvm:5173/api/v1/session/redirect/experiments/?experiment_id=0195c7fa-164e-7ede-9425-73e40a6f6e59&dataset_id=0195c7f9-76bd-7048-865e-2d8e3a7ef6c8&path=aHR0cDovL3RyYWluaW5ndm06NTE3My9hcGkv\in your Opik dashboard]8;;\.